In [1]:
from LAC import  LAC
import synonyms
import http.client
import hashlib
import urllib
import random
import json
import pandas as pd
import numpy as np
import time
import difflib

ModuleNotFoundError: No module named 'LAC'

In [2]:
#xlsx转csv
def xlsx_to_csv_pd(path1,path2):
    data_xls = pd.read_excel(path1+'.xlsx', index_col=0)
    data_xls.to_csv(path2+'.csv', encoding='utf-8')
#csv转xlsx
def csv_to_xlsx_pd(path1,path2):
    csv = pd.read_csv(path1+'.csv', encoding='utf-8')
    csv.to_excel(path2+'.xlsx', sheet_name='data')

In [91]:
#百度通用翻译API
# coding=utf-8
def translate(q):
    appid = '20220406001160200'  # 填写你的appid
    secretKey = 'hYUtVR6JR9kulHwTH3ap'  # 填写你的密钥

    httpClient = None
    myurl = '/api/trans/vip/translate'

    fromLang = 'auto'   #原文语种
    toLang = 'zh'   #译文语种
    salt = random.randint(32768, 65536)
    # q= 'apple'
    sign = appid + q + str(salt) + secretKey
    sign = hashlib.md5(sign.encode()).hexdigest()
    myurl = myurl + '?appid=' + appid + '&q=' + urllib.parse.quote(q) + '&from=' + fromLang + '&to=' + toLang + '&salt=' + str(
    salt) + '&sign=' + sign

    try:
        httpClient = http.client.HTTPConnection('api.fanyi.baidu.com')
        httpClient.request('GET', myurl)

        # response是HTTPResponse对象
        response = httpClient.getresponse()
        result_all = response.read().decode("utf-8")
        result = json.loads(result_all)
        # print('翻译：：')
        # print (result['trans_result'][0]['dst'])
        # print (type(result['trans_result']))
    except Exception as e:
        print (e)
    finally:
        if httpClient:
            httpClient.close()
    return result['trans_result'][0]['dst']
    # return  result
def translate_en(q):
    appid = '20220406001160200'  # 填写你的appid
    secretKey = 'hYUtVR6JR9kulHwTH3ap'  # 填写你的密钥

    httpClient = None
    myurl = '/api/trans/vip/translate'

    fromLang = 'zh'   #原文语种
    toLang = 'en'   #译文语种
    salt = random.randint(32768, 65536)
    # q= 'apple'
    sign = appid + q + str(salt) + secretKey
    sign = hashlib.md5(sign.encode()).hexdigest()
    myurl = myurl + '?appid=' + appid + '&q=' + urllib.parse.quote(q) + '&from=' + fromLang + '&to=' + toLang + '&salt=' + str(
    salt) + '&sign=' + sign

    try:
        httpClient = http.client.HTTPConnection('api.fanyi.baidu.com')
        httpClient.request('GET', myurl)

        # response是HTTPResponse对象
        response = httpClient.getresponse()
        result_all = response.read().decode("utf-8")
        result = json.loads(result_all)
        # print('翻译：：')
        # print (result['trans_result'][0]['dst'])
        # print (type(result['trans_result']))
    except Exception as e:
        print (e)
    finally:
        if httpClient:
            httpClient.close()
    return result['trans_result'][0]['dst']

In [92]:
https = "https://fanyi-api.baidu.com/api/trans/vip/translate"
result = translate_en('苹果')
result

'Apple'

In [100]:
name_list = ['阿拉维斯足球俱乐部','埃尔切足球俱乐部','奥萨苏纳足球俱乐部','巴塞罗那足球俱乐部','比利亚雷亚尔足球俱乐部','毕尔巴鄂竞技足球俱乐部','格拉纳达足球俱乐部','赫塔费足球俱乐部','皇家贝蒂斯足球俱乐部','皇家比戈塞尔塔足球俱乐部','皇家马德里足球俱乐部','皇家马略卡足球俱乐部','皇家社会足球俱乐部','皇家西班牙人足球俱乐部','加的斯足球俱乐部','莱万特足球俱乐部','马德里巴列卡诺闪电足球俱乐部','马德里竞技足球俱乐部','塞维利亚足球俱乐部','瓦伦西亚足球俱乐部']

In [60]:
#相似度度量对比,交换
def similar_diff_ratio(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()
def max_index(list):
    temp = list[0]
    index = 0
    for i in range(len(list)):
        if(list[i]>temp):
            temp = list[i]
            index = i
    return index
#list1是百度中文名，list2是领域知识库翻译后中文名
def simi_name(list1,list2):
    simi = []
    temp = []
    maxindex = [] #正确的下标
    # final_name = [] #更新后对应顺序名
    for i in range(len(list1)):
        for j in range(len(list2)):
            temp.append(similar_diff_ratio(list1[i],list2[j]))
        simi.append(temp)
        maxindex.append(max_index(temp))
        temp = []
    # for i in range(len(list1)):
    #     for j in range(len(max_index)):
    #         if(max_index[j]==i):
    #             final_name.append(list2(max_index[j]))
    return maxindex


In [40]:
#翻译中文
def transname(file1):
    df1 = pd.read_csv(file1)
    en_df = np.array(df1['Unnamed: 0'][1:])
    for i in range(len(en_df)):
        en_df[i] = en_df[i].split("\\")[0]
    en_name = list(en_df)
    zh_name = []
    for i in range(len(en_name)):
        zh_name.append(translate(en_name[i]))
        time.sleep(1)
    return zh_name

In [93]:
#加入英文名
#en_FBREFname,zh_FBREFname均为某支球队信息
def FBref_name_append(file1,zh_FBREFname,file2,index):
    df1 = pd.read_csv(file1)
    en_df = np.array(df1['Unnamed: 0'][1:])
    for i in range(len(en_df)):
        en_df[i] = en_df[i].split("\\")[0]
    en_FBREFname = list(en_df)
    df2 = pd.read_csv(file2,encoding='utf-8')
    df3 = df2[['球衣号码','姓名','国籍','生日','职位','所属球队']]
    df3['英文名'] = [' '] * df3.shape[0]
        # df3.reset_index(drop=True, inplace=True)
    zh_name = list(np.array(df2['姓名']))
    # print(zh_name)
    # print(zh_FBREFname[i])
    maxindex = simi_name(zh_name,zh_FBREFname[index])
    for i in range(len(maxindex)):
        for j in range(len(maxindex)):
            if(maxindex[j]==i):
                df3['英文名'][i] = en_FBREFname[j]
    for i in range(len(df3)):
        if(df3['英文名'][i]==' '):
            time.sleep(1)
            df3['英文名'][i] = translate_en(df3['姓名'][i])
    return df3


In [63]:
#领域知识库翻译
FBREF_zh1 = []
for i in range(0,5):
    file1 = "FBREF_player//"+name_list[i]+".csv"
    FBREF_zh1.append(transname(file1))
FBREF_zh1

In [64]:
#领域知识库翻译
FBREF_zh2 = []
for i in range(5,10):
    file1 = "FBREF_player//"+name_list[i]+".csv"
    FBREF_zh2.append(transname(file1))
FBREF_zh2

In [65]:
# 领域知识库翻译
time.sleep(5)
FBREF_zh3 = []
for i in range(10,15):
    file1 = "FBREF_player//"+name_list[i]+".csv"
    FBREF_zh3.append(transname(file1))
FBREF_zh3

In [66]:
# 领域知识库翻译
time.sleep(5)
FBREF_zh4 = []
for i in range(15,20):
    file1 = "FBREF_player//"+name_list[i]+".csv"
    FBREF_zh4.append(transname(file1))
FBREF_zh4

In [67]:
FBREF_zh = []
for i in range(5):
    FBREF_zh.append(FBREF_zh1[i])
for i in range(5):
    FBREF_zh.append(FBREF_zh2[i])
for i in range(5):
    FBREF_zh.append(FBREF_zh3[i])
for i in range(5):
    FBREF_zh.append(FBREF_zh4[i])


In [102]:
for i in range(8,9):
    file1 = "FBREF_player//"+name_list[i]+".csv"
    file2 = "球员xls//csv//"+name_list[i]+".csv"
    df = FBref_name_append(file1,FBREF_zh,file2,i)
    df.to_csv('球员xls//csv2//'+name_list[i]+'.csv')
    print(df)

<ipython-input-93-d7164947c3e2>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['英文名'][i] = en_FBREFname[j]
<ipython-input-93-d7164947c3e2>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['英文名'][i] = translate_en(df3['姓名'][i])


    球衣号码          姓名    国籍          生日  职位          所属球队               英文名
0      1    霍埃尔·罗夫莱斯   西班牙   1990.6.17  门将  皇家贝蒂斯足球俱乐部球员           Rodrigo
1      2      马丁·蒙托亚   西班牙   1991.4.14  后卫            球员    Martin Montoya
2      3    埃德加·冈萨雷斯   西班牙    1997.4.1  后卫    皇家贝蒂斯足球俱乐部    Edgar Gonzalez
3      4     保罗·阿库奥库  科特迪瓦  1997.12.20  中场           NaN       Nabil Fekir
4      5      马克·巴特拉   西班牙   1991.1.15  后卫           NaN        Mark batra
5      6     比克托·鲁伊斯   西班牙   1989.1.25  后卫           NaN       Bictor Ruiz
6      7         胡安米   西班牙   1993.5.20  前锋           NaN      Willian José
7      8     纳比勒·费基尔    法国   1993.7.18  中场           NaN      Aitor Ruibal
8      9  博尔哈·伊格莱西亚斯   西班牙   1993.1.17  前锋           NaN   Andrés Guardado
9     10   塞尔希奥·卡纳莱斯   西班牙   1991.2.16  中场           NaN   Youssouf Sabaly
10    11    克里斯蒂安·特略   西班牙   1991.8.11  前锋           NaN      Kike Hermoso
11    12       威廉·若泽    巴西  1991.11.23  前锋           NaN            Juanmi
12    13      鲁伊·席尔瓦   葡萄

In [88]:
if(df['英文名'][12]==' '):
    print(1)
df['英文名'][12]

1


' '

In [73]:
# df
FBREF_zh

[['路易斯·里奥哈',
  '约瑟卢',
  '维克多·拉瓜迪亚',
  '费尔南多·帕切科',
  '鲁本·杜阿尔特',
  '勒祖尼',
  '卢姆恩迪亚耶酒店',
  '马丁·阿格里加比尔',
  '托尼·莫亚',
  '托马斯皮纳岛',
  '佩雷庞斯酒店',
  '埃德加·门德斯',
  '马特·米兹加',
  '冈萨罗-埃斯卡兰特',
  '西莫·纳瓦罗',
  '马努加西亚',
  '纳胡尔·特纳格利亚',
  '米格尔',
  '手机',
  '杰森',
  '哈维·洛佩斯',
  '马马杜·塞拉',
  '约翰·吉德蒂',
  '多知',
  '安东尼奥·西维拉',
  '伊凡·马丁',
  '马努瓦列霍',
  '耶稣奥沃诺',
  '米尔恰·亚历山大·泰尔',
  '索尔·加西亚',
  'Abdelkabir Abqar',
  '亚历克斯·巴尔博亚',
  '艾伦·戈多伊',
  '太极原',
  '奥斯卡·梅迪纳',
  '托马斯·门德斯',
  '卢卡斯·佩雷斯',
  '阿德里安·罗德里格斯',
  '伊万·桑兹',
  '小队总数',
  '对手总数'],
 ['莫西卡',
  '奥马尔·马斯卡雷尔',
  '菲德尔',
  '恩佐·罗科',
  '卢卡斯·博伊',
  '海利贝尔顿·帕拉西奥斯',
  '埃德加·巴迪亚',
  '迭戈·冈萨雷斯',
  '卢卡斯·佩雷斯',
  '泰特莫伦特酒店',
  '杰拉德·甘博',
  '佩德罗·比加斯',
  '佩雷米拉',
  '劳尔·古蒂',
  '冈萨洛·维杜',
  '木子盒',
  '安东尼奥·巴拉甘',
  '伊万·马科内',
  '吉多·卡里略',
  '乔桑',
  '达里奥·贝内代托',
  '帕斯托雷',
  '以西奎尔庞塞',
  '皮亚蒂',
  '乔塞马',
  '卢卡斯·奥拉扎',
  '踢佩雷斯',
  '乔尼·拉莫',
  '安德烈',
  '塞尔吉奥·博诺·胡安',
  '南非',
  '凯撒',
  '约翰·唐纳德',
  '何塞·弗里扎',
  '框架底',
  '阿克塞尔·沃纳',
  '小队总数',
  '对手总数'],
 ['塞尔吉奥·赫雷拉',
  '大卫·加西亚',
  '乔恩·蒙卡约拉',
  '纳乔·维达尔',
  '卢卡斯·托罗',

In [38]:
file1 = "FBREF_player//"+name_list[0]+".csv"

file2 = "球员xls//csv//"+name_list[0]+".csv"
df,baidu,zh = FBref_name_append(file1,file2)
# df['name'][0]
# type(baidu)
baidu

['费尔南多·帕切科',
 '塔奇',
 '鲁文·杜阿尔特',
 '马特·米亚兹加',
 '比克托·拉瓜迪亚',
 '马马杜·卢姆',
 '马马杜·西拉',
 '托马斯·皮纳',
 '何塞卢',
 '约翰·圭代蒂',
 '路易斯·里奥哈',
 '萨乌尔·加西亚',
 '安东尼奥·西韦拉',
 '马努·加西亚',
 '托尼·莫亚',
 '埃德加·门德斯',
 '法昆多·佩利斯特里',
 '伊万·马丁',
 '佩雷·庞斯',
 '马丁·阿吉雷加维里亚',
 '弗洛里安·勒热纳',
 '西莫·纳瓦罗',
 '米格尔·德拉富恩特',
 '阿卜杜勒·阿卜卡尔',
 '哈维·洛佩斯',
 '赫苏斯·奥沃诺',
 '阿兰·戈多伊']

## File R/W

In [101]:
#转csv
for i in range(len(name_list)):
    xlsx_to_csv_pd('球员xls//球员xls//'+name_list[i],'球员xls//csv//'+name_list[i])

In [103]:
#转xlsx
for i in range(len(name_list)):
    csv_to_xlsx_pd('球员xls//csv2//'+name_list[i],'球员xls//xls//'+name_list[i])